# 02 — SQL Processing

Loads all raw inputs into DuckDB and produces a single tract-level master table ready for index construction.

Three joins to make:
- POI counts per tract (point-in-polygon)
- MBTA stop counts per tract (point-in-polygon)
- ACS demographics (attribute join on GEOID)

Output: `../data/processed/master_tracts.csv`

In [1]:
import geopandas as gpd
import pandas as pd
import numpy as np
import duckdb
import warnings
from pathlib import Path

warnings.filterwarnings('ignore')

RAW_DIR = Path('../data/raw')
PROCESSED_DIR = Path('../data/processed')
PROCESSED_DIR.mkdir(parents=True, exist_ok=True)

print('DuckDB version:', duckdb.__version__)

DuckDB version: 1.4.4


## 1. Load raw files


In [2]:
tracts = gpd.read_file(RAW_DIR / 'boston_tracts.gpkg')
stops = gpd.read_file(RAW_DIR / 'mbta_stops_boston.gpkg')
pois = gpd.read_file(RAW_DIR / 'boston_pois.gpkg')
acs = pd.read_csv(RAW_DIR / 'acs_demographics.csv', dtype={'GEOID': str})

print(f'Tracts:  {len(tracts)}')
print(f'Stops:   {len(stops)}')
print(f'POIs:    {len(pois)}')
print(f'ACS:     {len(acs)}')

Tracts:  205
Stops:   373
POIs:    3847
ACS:     235


## 2. Spatial joins in GeoPandas

Assigning each POI and each stop to its containing tract. Using `predicate='within'` — cleaner than `intersects` for point-in-polygon since points on boundaries won't double-count.

Result is a flat table of `(GEOID, poi_type)` rows and `(GEOID, transit_mode)` rows, which DuckDB will then aggregate.

In [3]:
# Reproject everything to the same CRS before joining
tracts = tracts.to_crs(epsg=4326)
stops = stops.to_crs(epsg=4326)
pois = pois.to_crs(epsg=4326)

# Keep only what we need from tracts for the join
tracts_slim = tracts[['GEOID', 'geometry']].copy()

# POIs → tracts
pois_joined = gpd.sjoin(
    pois[['poi_type', 'geometry']],
    tracts_slim,
    how='inner',
    predicate='within'
).drop(columns='index_right')

# Stops → tracts
stops_joined = gpd.sjoin(
    stops[['stop_id', 'transit_mode', 'geometry']],
    tracts_slim,
    how='inner',
    predicate='within'
).drop(columns='index_right')

print(f'POIs matched to tracts:  {len(pois_joined)}')
print(f'Stops matched to tracts: {len(stops_joined)}')

POIs matched to tracts:  3836
Stops matched to tracts: 371


## 3. Aggregation and joining in DuckDB

Three queries:
1. Aggregate POI counts per tract (total + by type)
2. Aggregate stop counts per tract (total + by mode)
3. Join everything together with ACS on GEOID

In [4]:
con = duckdb.connect()

# Register DataFrames as virtual tables
con.register('pois_raw', pois_joined[['GEOID', 'poi_type']])
con.register('stops_raw', stops_joined[['GEOID', 'transit_mode']])
con.register('acs_raw', acs)
con.register('tracts_raw', tracts[['GEOID']])

print('Tables registered:', con.execute("SHOW TABLES").fetchdf()['name'].tolist())

Tables registered: ['acs_raw', 'pois_raw', 'stops_raw', 'tracts_raw']


In [5]:
# --- Query 1: POI counts per tract ---
poi_counts = con.execute("""
    SELECT
        GEOID,
        COUNT(*) AS total_pois,
        COUNT(*) FILTER (WHERE poi_type = 'retail')        AS pois_retail,
        COUNT(*) FILTER (WHERE poi_type = 'food_beverage') AS pois_food_beverage,
        COUNT(*) FILTER (WHERE poi_type = 'services')      AS pois_services,
        COUNT(*) FILTER (WHERE poi_type = 'office')        AS pois_office
    FROM pois_raw
    GROUP BY GEOID
""").fetchdf()

print(f'Tracts with at least 1 POI: {len(poi_counts)}')
poi_counts.describe()

Tracts with at least 1 POI: 182


,total_pois,pois_retail,pois_food_beverage,pois_services,pois_office
count,182.000000,182.000000,182.000000,182.000000,182.000000
mean,21.076923,8.543956,8.736264,1.186813,2.609890
std,34.726152,17.629262,13.927861,2.526816,5.136427
min,1.000000,0.000000,0.000000,0.000000,0.000000
25%,3.000000,1.000000,1.000000,0.000000,0.000000
50%,9.000000,3.000000,4.000000,0.000000,1.000000
75%,22.000000,8.750000,10.750000,2.000000,3.000000
max,236.000000,153.000000,96.000000,22.000000,36.000000


In [6]:
# --- Query 2: Stop counts per tract ---
stop_counts = con.execute("""
    SELECT
        GEOID,
        COUNT(*)                                                    AS total_stops,
        COUNT(*) FILTER (WHERE transit_mode = 'subway')            AS stops_subway,
        COUNT(*) FILTER (WHERE transit_mode = 'light_rail')        AS stops_light_rail,
        COUNT(*) FILTER (WHERE transit_mode = 'bus')               AS stops_bus,
        -- Rapid transit flag: 1 if tract has any subway or light rail stop
        MAX(CASE WHEN transit_mode IN ('subway', 'light_rail') THEN 1 ELSE 0 END) AS has_rapid_transit
    FROM stops_raw
    GROUP BY GEOID
""").fetchdf()

print(f'Tracts with at least 1 stop: {len(stop_counts)}')
stop_counts.describe()

Tracts with at least 1 stop: 84


,total_stops,stops_subway,stops_light_rail,stops_bus,has_rapid_transit
count,84.000000,84.000000,84.000000,84.000000,84.000000
mean,4.416667,0.869048,1.035714,2.511905,0.571429
std,4.335920,1.641185,1.745538,2.262816,0.497844
min,1.000000,0.000000,0.000000,0.000000,0.000000
25%,1.000000,0.000000,0.000000,1.000000,0.000000
50%,3.000000,0.000000,0.000000,2.000000,1.000000
75%,6.000000,2.000000,2.000000,3.000000,1.000000
max,30.000000,10.000000,7.000000,16.000000,1.000000


In [7]:
# --- Query 3: Master join ---
# Start from all tracts (left join) so tracts with zero POIs or stops
# are kept — they'll get 0s, not dropped

con.register('poi_counts', poi_counts)
con.register('stop_counts', stop_counts)

master = con.execute("""
    SELECT
        t.GEOID,

        -- Demographics
        a.median_hh_income,
        a.total_population,
        a.pop_25_54,
        a.share_25_54,

        -- Transit
        COALESCE(s.total_stops,      0) AS total_stops,
        COALESCE(s.stops_subway,     0) AS stops_subway,
        COALESCE(s.stops_light_rail, 0) AS stops_light_rail,
        COALESCE(s.stops_bus,        0) AS stops_bus,
        COALESCE(s.has_rapid_transit,0) AS has_rapid_transit,

        -- POIs
        COALESCE(p.total_pois,          0) AS total_pois,
        COALESCE(p.pois_retail,         0) AS pois_retail,
        COALESCE(p.pois_food_beverage,  0) AS pois_food_beverage,
        COALESCE(p.pois_services,       0) AS pois_services,
        COALESCE(p.pois_office,         0) AS pois_office

    FROM tracts_raw t
    LEFT JOIN acs_raw        a ON t.GEOID = a.GEOID
    LEFT JOIN stop_counts    s ON t.GEOID = s.GEOID
    LEFT JOIN poi_counts     p ON t.GEOID = p.GEOID

    ORDER BY t.GEOID
""").fetchdf()

print(f'Master table rows: {len(master)}')
print(f'Columns: {list(master.columns)}')
master.head()

Master table rows: 205
Columns: ['GEOID', 'median_hh_income', 'total_population', 'pop_25_54', 'share_25_54', 'total_stops', 'stops_subway', 'stops_light_rail', 'stops_bus', 'has_rapid_transit', 'total_pois', 'pois_retail', 'pois_food_beverage', 'pois_services', 'pois_office']


,GEOID,median_hh_income,total_population,pop_25_54,share_25_54,total_stops,stops_subway,stops_light_rail,stops_bus,has_rapid_transit,total_pois,pois_retail,pois_food_beverage,pois_services,pois_office
0,25025000101,135156.0,1734,875,0.504614,0,0,0,0,0,7,2,1,2,2
1,25025000102,84044.0,3979,1869,0.469716,1,0,0,1,0,52,21,17,4,10
2,25025000201,111477.0,4375,2506,0.572800,0,0,0,0,0,9,3,6,0,0
3,25025000202,84792.0,3892,1646,0.422919,0,0,0,0,0,16,4,6,6,0
4,25025000301,133051.0,2719,1310,0.481795,0,0,0,0,0,9,4,4,0,1


## 4. Data quality checks

Checking for nulls and zero-POI tracts before moving to scoring. A tract with null income is fine — we'll handle it in normalisation. A tract with zero POIs and zero stops is probably a park or industrial area and will naturally score low.

In [8]:
print('=== Null counts ===')
print(master.isnull().sum())

print('\n=== Zero-POI tracts ===')
print(f"  {(master['total_pois'] == 0).sum()} tracts have no POIs")

print('\n=== Zero-stop tracts ===')
print(f"  {(master['total_stops'] == 0).sum()} tracts have no transit stops")

print('\n=== Population range ===')
print(master['total_population'].describe())

=== Null counts ===
GEOID                  0
median_hh_income      14
total_population       0
pop_25_54              0
share_25_54            5
total_stops            0
stops_subway           0
stops_light_rail       0
stops_bus              0
has_rapid_transit      0
total_pois             0
pois_retail            0
pois_food_beverage     0
pois_services          0
pois_office            0
dtype: int64

=== Zero-POI tracts ===
  23 tracts have no POIs

=== Zero-stop tracts ===
  121 tracts have no transit stops

=== Population range ===
count     205.000000
mean     3248.512195
std      1670.226422
min         0.000000
25%      2235.000000
50%      3117.000000
75%      4233.000000
max      9131.000000
Name: total_population, dtype: float64


In [9]:
# Fill zero-population tracts with NaN for income/age ratios
# (avoids divide-by-zero artefacts in scoring)
master.loc[master['total_population'] == 0, ['median_hh_income', 'share_25_54']] = np.nan

print('Nulls after population fix:')
print(master[['median_hh_income', 'share_25_54']].isnull().sum())

Nulls after population fix:
median_hh_income    14
share_25_54          5
dtype: int64


## 5. Save

In [10]:
master.to_csv(PROCESSED_DIR / 'master_tracts.csv', index=False)
print(f'Saved: master_tracts.csv ({len(master)} rows x {len(master.columns)} cols)')
print(f'Location: {(PROCESSED_DIR / "master_tracts.csv").resolve()}')
master.describe()

Saved: master_tracts.csv (205 rows x 15 cols)
Location: /Users/administrator/Documents/project 3/data/processed/master_tracts.csv


,median_hh_income,total_population,pop_25_54,share_25_54,total_stops,stops_subway,stops_light_rail,stops_bus,has_rapid_transit,total_pois,pois_retail,pois_food_beverage,pois_services,pois_office
count,191.000000,205.000000,205.000000,200.000000,205.000000,205.000000,205.000000,205.000000,205.000000,205.000000,205.000000,205.000000,205.000000,205.000000
mean,96157.287958,3248.512195,1197.068293,0.387991,1.809756,0.356098,0.424390,1.029268,0.234146,18.712195,7.585366,7.756098,1.053659,2.317073
std,48386.554759,1670.226422,671.740113,0.145472,3.519957,1.131121,1.224901,1.901785,0.424501,33.382808,16.824317,13.407235,2.409550,4.908172
min,18125.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,60611.000000,2235.000000,758.000000,0.291588,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000,0.000000,0.000000,0.000000,0.000000
50%,86250.000000,3117.000000,1119.000000,0.374763,0.000000,0.000000,0.000000,0.000000,0.000000,8.000000,3.000000,3.000000,0.000000,1.000000
75%,127745.500000,4233.000000,1619.000000,0.462219,2.000000,0.000000,0.000000,1.000000,0.000000,19.000000,7.000000,8.000000,1.000000,2.000000
max,245000.000000,9131.000000,2972.000000,1.000000,30.000000,10.000000,7.000000,16.000000,1.000000,236.000000,153.000000,96.000000,22.000000,36.000000


In [11]:
con.close()
print('DuckDB connection closed.')

DuckDB connection closed.
